In [15]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials
import io
import os
import requests
import calendar

In [16]:
@task(retries=3)
def extract_load_web_data(year:int , month:int,file_name) -> None:
    request_url =f"https://use-land-property-data.service.gov.uk/datasets/td/download/history/{calendar.month_name[month]}%20{year}/{file_name}.csv"
    r = requests.get(request_url)
    url_content=r.content
    csv_file = open(f'{file_name}.csv', 'wb')
    csv_file.write(url_content)
    csv_file.close()

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'extract_load_web_data' and defined at '/tmp/ipykernel_1015847/1744432909.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [17]:
@task()
def load_csv_data (file_name: str) -> pd.DataFrame:
    df = pd.read_csv(f"{file_name}.csv", sep=',', header=4, index_col=False)
    return df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'load_csv_data' and defined at '/tmp/ipykernel_1015847/3339839138.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [18]:
@task()
def clean_local_data(df: pd.DataFrame) -> pd.DataFrame:
    """ We need to add a date column for each DataFrame """
    new_df = df[df["Account Customer"].str.contains("Total")==False]
    """ We are dropping the "Total" Column from the Dataframe """
    new_df = new_df.drop(["Total"], axis=1)
    new_df = new_df.columns.str.replace("[()]", "_")
    return new_df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'clean_local_data' and defined at '/tmp/ipykernel_1015847/2651271778.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [19]:
@task()
def transform_data(df: pd.DataFrame, year, month) -> pd.DataFrame:
    df["date_added"] = f"01.{month:02}.{year}"
    return df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'transform_data' and defined at '/tmp/ipykernel_1015847/2430716275.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [20]:
@task()
def save_transformed_data(df: pd.DataFrame, file_name) -> None:
    df.to_parquet(f"{file_name}.parquet", compression="gzip", index=False)
    df = pd.read_parquet(f"{file_name}.parquet")
    return df

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'save_transformed_data' and defined at '/tmp/ipykernel_1015847/4246081314.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [21]:
@task()
def write_gcs(file_name: str, path_gcp) -> None:
    """Upload local parquet FIle """
    gcp_cloud_storage_bucket_block = GcsBucket.load("gcp-project-dc")
    gcp_cloud_storage_bucket_block.upload_from_path(
        from_path = f"{file_name}.parquet",
        to_path=f"{path_gcp}/{file_name}"
    )

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'write_gcs' and defined at '/tmp/ipykernel_1015847/1425389079.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [22]:
@task()
def write_bq(df: pd.DataFrame) -> None:
    """Write DataFrame to BigQuery"""
    gcp_credentials_block = GcpCredentials.load("gcp-project-2023")
    df.to_gbq(
        destination_table="project_dwh.land_and_property", # Dataset.TableName
        project_id="dc-project-2023",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        chunksize=500.000,
        if_exists="append",
    )

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'write_bq' and defined at '/tmp/ipykernel_1015847/1188040828.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [23]:
@flow()
def etl_web_to_local_to_gcs(year: int, month: int) -> None:
    file_name = f"Number-and-types-of-applications-by-all-account-customers-{year}-{month:02}"
    extract_load_web_data(year, month, file_name)
    raw_df = load_csv_data(file_name)
    cleaned_df = clean_local_data(raw_df)
    transformed_df = transform_data (cleaned_df, year, month)
    final_df = save_transformed_data (transformed_df, file_name)
    write_gcs(file_name, "Land_and_Property")
    write_bq(final_df)
    

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-web-to-local-to-gcs' and defined at '/tmp/ipykernel_1015847/506873230.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [24]:
@flow()
def etl_parent_flow(years:list[int] , months:list[int]) -> None:
    for year in years:
        for month in months:
            etl_web_to_local_to_gcs(year, month)

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-parent-flow' and defined at '/tmp/ipykernel_1015847/1325823985.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


## Execution

In [25]:
if __name__ == '__main__':
    years=[2022]
    months=[2,3]
    etl_parent_flow(years, months) 

12:54:30.188 | INFO    | prefect.engine - Created flow run 'happy-dingo' for flow 'etl-parent-flow'

/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py:261: UserWarning: A task named 'etl-web-to-local-to-gcs' and defined at '/tmp/ipykernel_1015847/506873230.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


12:54:30.484 | INFO    | Flow run 'happy-dingo' - Created subflow run 'eggplant-woodpecker' for flow 'etl-web-to-local-to-gcs'

12:54:30.601 | INFO    | Flow run 'eggplant-woodpecker' - Created task run 'extract_load_web_data-0' for task 'extract_load_web_data'

12:54:30.604 | INFO    | Flow run 'eggplant-woodpecker' - Executing 'extract_load_web_data-0' immediately...

...

12:54:37.669 | ERROR   | Flow run 'eggplant-woodpecker' - Encountered exception during execution:
Traceback (most recent call last):
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 642, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 154, in run_sync_in_interruptible_worker_thread
    async with anyio.create_task_group() as tg:
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/tmp/ipykernel_1015847/506873230.py", line 7, in etl_web_to_local_to_gcs
    transformed_df = transform_data (cleaned_df, year, month)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/tasks.py", line 448, in __call__
    return enter_task_run_engine(
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 932, in enter_task_run_engine
    return run_async_from_worker_thread(begin_run)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 458, in result
    return self.__get_result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._exception
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 1081, in get_task_call_return_value
    return await future._result()
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/futures.py", line 237, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/states.py", line 89, in _get_state_result
    raise await get_state_exception(state)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 1487, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 154, in run_sync_in_interruptible_worker_thread
    async with anyio.create_task_group() as tg:
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/

12:54:37.732 | ERROR   | Flow run 'eggplant-woodpecker' - Finished in state Failed('Flow run encountered an exception. TypeError: Index does not support mutable operations\n')

12:54:37.736 | ERROR   | Flow run 'happy-dingo' - Encountered exception during execution:
Traceback (most recent call last):
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 642, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 154, in run_sync_in_interruptible_worker_thread
    async with anyio.create_task_group() as tg:
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/tmp/ipykernel_1015847/1325823985.py", line 5, in etl_parent_flow
    etl_web_to_local_to_gcs(year, month)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/flows.py", line 448, in __call__
    return enter_flow_run_engine_from_flow_call(
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 174, in enter_flow_run_engine_from_flow_call
    return run_async_from_worker_thread(begin_run)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 458, in result
    return self.__get_result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._exception
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/client/utilities.py", line 47, in with_injected_client
    return await fn(*args, **kwargs)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 552, in create_and_begin_subflow_run
    return await terminal_state.result(fetch=True)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/states.py", line 89, in _get_state_result
    raise await get_state_exception(state)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/engine.py", line 642, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", line 154, in run_sync_in_interruptible_worker_thread
    async with anyio.create_task_group() as tg:
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/home/linux_dev_env/py_env/lib/python3.10/site-packages/prefect/utilities/asyncutils.py", lin

12:54:37.787 | ERROR   | Flow run 'happy-dingo' - Finished in state Failed('Flow run encountered an exception. TypeError: Index does not support mutable operations\n')

TypeError: Index does not support mutable operations